In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
!pip install --upgrade xlrd #colab bug verson xlrd
!pip install geopandas
!pip install pandas fiona shapely pyproj rtree
!pip install contextily
!pip install pygeos
!pip install topojson
import geopandas as gpd
import contextily as ctx
import matplotlib.pyplot as plt
#!pip install requests py7zr geopandas openpyxl tqdm s3fs PyYAML xlrd
#!pip install git+https://github.com/inseefrlab/cartiflette@80b8a5a28371feb6df31d55bcc2617948a5f9b1a

In [206]:
data_pollution = pd.read_csv("https://koumoul.com/data-fair/api/v1/datasets/igt-pouvoir-de-rechauffement-global/convert", encoding = 'utf-8')

In [225]:
base_geo_pour_merge = pd.read_csv("https://www.insee.fr/fr/statistiques/fichier/6800675/v_commune_2023.csv", encoding = 'utf-8')
#base_geo_pour_merge_2 = base_geo_pour_merge['DEP'] and base_geo_pour_merge['REG']
base_geo_pour_merge

,TYPECOM,COM,REG,DEP,CTCD,ARR,TNCC,NCC,NCCENR,LIBELLE,CAN,COMPARENT
0,COM,01001,84.0,01,01D,012,5,ABERGEMENT CLEMENCIAT,Abergement-Clémenciat,L'Abergement-Clémenciat,0108,NaN
1,COM,01002,84.0,01,01D,011,5,ABERGEMENT DE VAREY,Abergement-de-Varey,L'Abergement-de-Varey,0101,NaN
2,COM,01004,84.0,01,01D,011,1,AMBERIEU EN BUGEY,Ambérieu-en-Bugey,Ambérieu-en-Bugey,0101,NaN
3,COM,01005,84.0,01,01D,012,1,AMBERIEUX EN DOMBES,Ambérieux-en-Dombes,Ambérieux-en-Dombes,0122,NaN
4,COM,01006,84.0,01,01D,011,1,AMBLEON,Ambléon,Ambléon,0104,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
37558,COM,97613,6.0,976,976D,NaN,0,M TSANGAMOUJI,M'Tsangamouji,M'Tsangamouji,97613,NaN
37559,COM,97614,6.0,976,976D,NaN,1,OUANGANI,Ouangani,Ouangani,97610,NaN
37560,COM,97615,6.0,976,976D,NaN,0,PAMANDZI,Pamandzi,Pamandzi,97611,NaN
37561,COM,97616,6.0,976,976D,NaN,0,SADA,Sada,Sada,97612,NaN


In [208]:
base_geo_pour_merge.drop("DEP", axis = 1, inplace = True)
base_geo_pour_merge.drop("REG", axis = 1, inplace = True)

In [209]:
data_pollution['DEP'] = data_pollution['INSEE commune'].str[:2]

In [210]:
correspondance_reg = {
    84 : {1, 3, 7, 15, 26, 38, 42, 43, 63, 69, 73, 74}, #Auvergne-Rhône-Alpes
    27 : {21, 25, 39, 58, 70, 71, 89, 90}, #Bourgogne-Franche-Comté
    53 : {22, 29, 35, 56}, #Bretagne
    28 : {18, 28, 36, 37, 41, 45}, #Centre-Val de Loire
    94 : {'2A', '2B'}, #Corse
    44 : {8, 10, 51, 52, 54, 55, 57, 67, 68, 88}, #Grand Est
    32 : {2, 59, 60, 62, 80}, #Hauts-de-France
    11 : {75, 77, 78, 91, 92, 93, 94, 95}, #Île-de-France
    76 : {14, 27, 50, 61, 76}, #Normandie
    75 : {16, 17, 19, 23, 24, 33, 40, 47, 64, 79, 86, 87}, #Nouvelle-Aquitaine
    24 : {9, 11, 12, 30, 31, 32, 34, 46, 48, 65, 66, 81, 82}, #Occitanie
    52 : {44, 49, 53, 72, 85}, #Pays de la Loire
    93 : {4, 5, 6, 13, 83, 84} #Provence-Alpes-Côte d'Azur
}

In [211]:
data_pollution['REG'] = data_pollution['DEP'].map(correspondance_reg)

In [213]:
base_gen_pollution = pd.merge(data_pollution, base_geo_pour_merge, how='inner', 
                              left_on='INSEE commune', right_on='COM')
base_gen_pollution

,INSEE commune,Commune,Agriculture,Autres transports,Autres transports international,CO2 biomasse hors-total,Déchets,Energie,Industrie hors-énergie,Résidentiel,...,TYPECOM,COM,CTCD,ARR,TNCC,NCC,NCCENR,LIBELLE,CAN,COMPARENT
0,01001,L'ABERGEMENT-CLEMENCIAT,3711.425991,NaN,NaN,432.751835,101.430476,2.354558,6.911213,309.358195,...,COM,01001,01D,012,5,ABERGEMENT CLEMENCIAT,Abergement-Clémenciat,L'Abergement-Clémenciat,0108,NaN
1,01002,L'ABERGEMENT-DE-VAREY,475.330205,NaN,NaN,140.741660,140.675439,2.354558,6.911213,104.866444,...,COM,01002,01D,011,5,ABERGEMENT DE VAREY,Abergement-de-Varey,L'Abergement-de-Varey,0101,NaN
2,01004,AMBERIEU-EN-BUGEY,499.043526,212.577908,NaN,10313.446515,5314.314445,998.332482,2930.354461,16616.822534,...,COM,01004,01D,011,1,AMBERIEU EN BUGEY,Ambérieu-en-Bugey,Ambérieu-en-Bugey,0101,NaN
3,01005,AMBERIEUX-EN-DOMBES,1859.160954,NaN,NaN,1144.429311,216.217508,94.182310,276.448534,663.683146,...,COM,01005,01D,012,1,AMBERIEUX EN DOMBES,Ambérieux-en-Dombes,Ambérieux-en-Dombes,0122,NaN
4,01006,AMBLEON,448.966808,NaN,NaN,77.033834,48.401549,NaN,NaN,43.714019,...,COM,01006,01D,011,1,AMBLEON,Ambléon,Ambléon,0104,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36191,95676,VILLERS-EN-ARTHIES,1628.065094,NaN,NaN,165.045396,65.063617,11.772789,34.556067,176.098160,...,COM,95676,95D,953,0,VILLERS EN ARTHIES,Villers-en-Arthies,Villers-en-Arthies,9520,NaN
36192,95678,VILLIERS-ADAM,698.630772,NaN,NaN,1331.126598,111.480954,2.354558,6.911213,1395.529811,...,COM,95678,95D,953,0,VILLIERS ADAM,Villiers-Adam,Villiers-Adam,9514,NaN
36193,95680,VILLIERS-LE-BEL,107.564967,NaN,NaN,8367.174532,225.622903,534.484607,1568.845431,22613.830247,...,COM,95680,95D,952,0,VILLIERS LE BEL,Villiers-le-Bel,Villiers-le-Bel,9521,NaN
36194,95682,VILLIERS-LE-SEC,1090.890170,NaN,NaN,326.748418,108.969749,2.354558,6.911213,67.235487,...,COM,95682,95D,952,0,VILLIERS LE SEC,Villiers-le-Sec,Villiers-le-Sec,9509,NaN


In [214]:
data_pollution.head()

,INSEE commune,Commune,Agriculture,Autres transports,Autres transports international,CO2 biomasse hors-total,Déchets,Energie,Industrie hors-énergie,Résidentiel,Routier,Tertiaire,DEP,REG
0,01001,L'ABERGEMENT-CLEMENCIAT,3711.425991,NaN,NaN,432.751835,101.430476,2.354558,6.911213,309.358195,793.156501,367.036172,01,NaN
1,01002,L'ABERGEMENT-DE-VAREY,475.330205,NaN,NaN,140.741660,140.675439,2.354558,6.911213,104.866444,348.997893,112.934207,01,NaN
2,01004,AMBERIEU-EN-BUGEY,499.043526,212.577908,NaN,10313.446515,5314.314445,998.332482,2930.354461,16616.822534,15642.420313,10732.376934,01,NaN
3,01005,AMBERIEUX-EN-DOMBES,1859.160954,NaN,NaN,1144.429311,216.217508,94.182310,276.448534,663.683146,1756.341319,782.404357,01,NaN
4,01006,AMBLEON,448.966808,NaN,NaN,77.033834,48.401549,NaN,NaN,43.714019,398.786800,51.681756,01,NaN


In [215]:
merging_list = data_pollution.columns

In [216]:
merging_list = list(merging_list)

In [217]:
print(merging_list)

['INSEE commune', 'Commune', 'Agriculture', 'Autres transports', 'Autres transports international', 'CO2 biomasse hors-total', 'Déchets', 'Energie', 'Industrie hors-énergie', 'Résidentiel', 'Routier', 'Tertiaire', 'DEP', 'REG']


In [218]:
base_gen_pollution = base_gen_pollution[merging_list]

In [219]:
base_gen_pollution.head()

,INSEE commune,Commune,Agriculture,Autres transports,Autres transports international,CO2 biomasse hors-total,Déchets,Energie,Industrie hors-énergie,Résidentiel,Routier,Tertiaire,DEP,REG
0,01001,L'ABERGEMENT-CLEMENCIAT,3711.425991,NaN,NaN,432.751835,101.430476,2.354558,6.911213,309.358195,793.156501,367.036172,01,NaN
1,01002,L'ABERGEMENT-DE-VAREY,475.330205,NaN,NaN,140.741660,140.675439,2.354558,6.911213,104.866444,348.997893,112.934207,01,NaN
2,01004,AMBERIEU-EN-BUGEY,499.043526,212.577908,NaN,10313.446515,5314.314445,998.332482,2930.354461,16616.822534,15642.420313,10732.376934,01,NaN
3,01005,AMBERIEUX-EN-DOMBES,1859.160954,NaN,NaN,1144.429311,216.217508,94.182310,276.448534,663.683146,1756.341319,782.404357,01,NaN
4,01006,AMBLEON,448.966808,NaN,NaN,77.033834,48.401549,NaN,NaN,43.714019,398.786800,51.681756,01,NaN


In [220]:
base_gen_pollution['REG'].unique()

array([nan], dtype=object)

In [61]:
base_gen_pollution.shape

(36196, 14)

In [134]:
len(base_gen_pollution['INSEE commune'].unique())

35626

In [ ]:
len(base_gen_pollution['REG'].unique())

In [143]:
len(base_gen_pollution['DEP'].unique())

97

In [16]:
#probleme pour plus tard
#differences avec le nombre de regions et de departement dans l autre table 
# Gabrielle veut bien s'en charger

In [161]:
# Filtrer les lignes pour lesquelles le département ou la région ne sont pas renseignés
#(ou au sens "au moins l'un des deux)
REG_DEP_na = data_pollution[(data_pollution['DEP'].isna())] #| (base_gen_pollution['REG'].isna())
REG_DEP_na.shape
#on perd les données de 1339 communes donc il faudrait trouver un moyen de prendre le département à partir du code commune

(0, 24)

In [76]:
tentative_groupes_1 = base_gen_pollution.drop(['Commune'], axis = 1).groupby('DEP').sum()

In [77]:
tentative_groupes_1

,INSEE commune,Agriculture,Autres transports,Autres transports international,CO2 biomasse hors-total,Déchets,Energie,Industrie hors-énergie,Résidentiel,Routier,Tertiaire,REG
DEP,,,,,,,,,,,,
01,0100101002010040100501006010070100801009010100...,7.816819e+05,15372.396872,2.581109e+02,7.031481e+05,270526.994907,107108.831646,704385.028742,5.464377e+05,1.599545e+06,412255.462975,32928.0
02,0200102002020030200402005020060200702008020090...,1.270742e+06,57904.229456,2.476900e+03,6.158998e+05,180007.937389,60344.960348,737257.067508,6.378827e+05,1.381081e+06,324154.207262,25536.0
03,0300103002030030300403005030060300703008030090...,1.947087e+06,19686.272920,5.451597e+02,5.657408e+05,111205.867274,102975.024821,457513.404309,4.456648e+05,1.164099e+06,224438.681860,26628.0
04,0400104004040050400604007040080400904012040130...,3.604105e+05,5851.619941,0.000000e+00,1.157754e+05,50497.301309,12184.836311,60899.229209,1.166402e+05,3.882993e+05,98069.382968,18414.0
05,0500105003050040500605007050080500905010050110...,3.003535e+05,6216.001726,0.000000e+00,8.372760e+04,22099.068688,5453.155729,16006.370120,1.215160e+05,3.441621e+05,77274.057096,15066.0
...,...,...,...,...,...,...,...,...,...,...,...,...
91,9100191016910179102191022910279103591037910389...,1.547301e+05,839441.486655,1.257594e+06,7.187767e+05,293175.383972,104318.680722,364886.004497,1.270247e+06,2.015019e+06,747241.748820,2134.0
92,9200292004920079200992012920149201992020920229...,9.140818e+01,12340.794839,2.101194e+02,1.067889e+06,264497.880711,242842.018012,706597.424067,1.466794e+06,1.198420e+06,836013.153414,396.0
93,9300193005930069300793008930109301393014930159...,2.018471e+03,59617.086124,1.101400e+06,7.259516e+05,252166.943778,102837.663903,433216.360990,1.316452e+06,1.396911e+06,863017.791777,440.0


In [72]:
tentative_groupes_1 = tentative_groupes_1.drop(['INSEE commune', 'REG'], axis = 1)

In [74]:
tentative_groupes_1

,Agriculture,Autres transports,Autres transports international,CO2 biomasse hors-total,Déchets,Energie,Industrie hors-énergie,Résidentiel,Routier,Tertiaire
DEP,,,,,,,,,,
01,7.816819e+05,15372.396872,2.581109e+02,7.031481e+05,270526.994907,107108.831646,704385.028742,5.464377e+05,1.599545e+06,412255.462975
02,1.270742e+06,57904.229456,2.476900e+03,6.158998e+05,180007.937389,60344.960348,737257.067508,6.378827e+05,1.381081e+06,324154.207262
03,1.947087e+06,19686.272920,5.451597e+02,5.657408e+05,111205.867274,102975.024821,457513.404309,4.456648e+05,1.164099e+06,224438.681860
04,3.604105e+05,5851.619941,0.000000e+00,1.157754e+05,50497.301309,12184.836311,60899.229209,1.166402e+05,3.882993e+05,98069.382968
05,3.003535e+05,6216.001726,0.000000e+00,8.372760e+04,22099.068688,5453.155729,16006.370120,1.215160e+05,3.441621e+05,77274.057096
...,...,...,...,...,...,...,...,...,...,...
91,1.547301e+05,839441.486655,1.257594e+06,7.187767e+05,293175.383972,104318.680722,364886.004497,1.270247e+06,2.015019e+06,747241.748820
92,9.140818e+01,12340.794839,2.101194e+02,1.067889e+06,264497.880711,242842.018012,706597.424067,1.466794e+06,1.198420e+06,836013.153414
93,2.018471e+03,59617.086124,1.101400e+06,7.259516e+05,252166.943778,102837.663903,433216.360990,1.316452e+06,1.396911e+06,863017.791777


In [21]:
tentative_groupes_1.shape

(96, 10)

In [22]:
set_dep_littoral = (59, 62, 80, 76, 27, 14, 50, 35, 
                      22, 29, 56, 44, 85, 17, 33, 40, 
                    64, 66, 11, 34,
                    30, 13, 83, 0o6, '2A', '2B')

In [23]:
print(set_dep_littoral)

(59, 62, 80, 76, 27, 14, 50, 35, 22, 29, 56, 44, 85, 17, 33, 40, 64, 66, 11, 34, 30, 13, 83, 6, '2A', '2B')


In [144]:
base_gen_pollution['DEP'].unique()

array(['01', nan, '02', '03', '04', '05', '06', '07', '08', '09', '10',
       '11', '12', '13', '14', '15', '16', '17', '18', '19', '21', '22',
       '23', '24', '25', '26', '27', '28', '29', '2A', '2B', '30', '31',
       '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42',
       '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53',
       '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64',
       '65', '66', '67', '68', '69', '70', '71', '72', '73', '74', '75',
       '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86',
       '87', '88', '89', '90', '91', '92', '93', '94', '95'], dtype=object)

In [25]:
#on a pas les DROM donc complique par rapport a l'autre base 

In [78]:
# Importation de la base en local pour faire tourner le code main
base_gen_pollution.to_csv('base_pollution.csv', sep=',', index=False, encoding='utf-8')